In [41]:
import pandas as pd
import re
import os

# ---- Учитај фајлове ----
df1 = pd.read_csv("basic_imsbs.csv")  
df2 = pd.read_csv("learning_imsbs10.csv")  

# ---- Извуци 4 броја из имена фајла ----
def extract_numbers(filename):
    # узимамо сва 4 броја из имена: n_m_k_index
    m = re.match(r".*?_(\d+)_(\d+)_(\d+)_(\d+)\.txt\.out", filename)
    if m:
        return int(m.group(1)), int(m.group(2)), int(m.group(3)), int(m.group(4))
    else:
        return None, None, None, None

for df in [df1, df2]:
    df[['n','m','k','idx']] = df['file'].apply(lambda x: pd.Series(extract_numbers(x)))
    # kreiramo key po prva tri broja
    df['key'] = df[['n','m','k']].astype(str).agg('_'.join, axis=1)

# ---- Spoji podatke po key ----
merged = pd.merge(df1, df2, on='key', suffixes=('_1','_2'))

# ---- Agregiraj po prva tri broja (key) ----
agg = merged.groupby('key').agg({
    'time_1': 'mean',
    'quality_1': 'mean',
    'time_2': 'mean',
    'quality_2': 'mean'
}).reset_index()

# ---- Rezultat ----
print(agg)

# ---- Sačuvaj u CSV ----
agg.to_csv("merged_aggregated.csv", index=False)


         key    time_1  quality_1     time_2  quality_2
0   10_100_2  0.010758        8.6   0.011859        8.6
1   10_100_4  0.015646        6.2   0.015234        6.2
2   10_200_2  0.017391       10.3   0.017956       10.3
3   10_200_4  0.016767        6.2   0.025147        6.1
4   10_500_2  0.009166        9.5   0.046565        9.3
5   10_500_4  0.018688        6.1   0.032619        6.1
6    10_50_2  0.013251        9.1   0.011734        9.1
7    10_50_4  0.014968        6.0   0.014100        6.0
8    2_100_2  0.074699       72.8   0.192868       72.7
9    2_100_4  0.076843       61.7   0.171048       61.7
10   2_200_2  0.329666      136.9   1.133805      139.3
11   2_200_4  0.394792      125.1   1.027308      120.1
12   2_500_2  3.156137      272.1  16.612360      276.7
13   2_500_4  3.025999      299.3  10.203167      276.5
14    2_50_2  0.006316       37.7   0.011470       37.7
15    2_50_4  0.014924       30.1   0.026941       30.0
16   3_100_2  0.229338       58.5   0.504950    

In [42]:
agg[["time_1", "time_2", "quality_1", "quality_2" ]].mean()

time_1        0.769833
time_2        3.026354
quality_1    60.431250
quality_2    59.190625
dtype: float64